# Компания: Почта России (https://www.pochta.ru/)

In [16]:
import requests
import json
import math

In [3]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"

## Калькулятор на сайте (https://otpravka.pochta.ru/calculator)

In [5]:
# получаем токен
url = "https://otpravka.pochta.ru/calculator"
headers = {"User-Agent": user_agent}

session = requests.Session()
resp = session.get(url=url, headers=headers)
resp.raise_for_status()
token = session.cookies.get_dict()["XSRF-TOKEN"]
token

'5c0d2f06-1a9f-4999-9a90-9310573efe5f'

In [11]:
# идентификация населённых пунктов
url = "https://otpravka.pochta.ru/api/calculator/places"
headers = {"User-Agent": user_agent,
           "X-XSRF-TOKEN": token}
input_data = {"query": "Казань"}
resp = session.get(url=url, params=input_data, headers=headers)
resp.raise_for_status()
output_data = resp.json()
output_data

[{'index': '420000', 'place': 'г Казань', 'region': 'Респ Татарстан'},
 {'area': 'р-н Балашовский',
  'index': '412334',
  'place': 'с Рассказань',
  'region': 'обл Саратовская'}]

In [22]:
# расчёт стоимости и сроков доставки
url = "https://otpravka.pochta.ru/api/calculator/calculate"
headers = {"User-Agent": user_agent,
           "Content-Type": "application/json;charset=UTF-8",
           "X-XSRF-TOKEN": token}
rates = {1: "Стандарт",
         2: "Интернет-магазины"}
weights = [5.5]  # вес в килограммах, можно указать несколько

for rate_code in rates:  # проходим по кодам тарифов...
    for pickup_variant in ("POST_OFFICE", "COURIER"):  # считаем от терминала и двери

        print(f"{rates[rate_code]} от {'терминала' if pickup_variant == 'POST_OFFICE' else 'двери'}")
        input_data = {"hasOnlineShop": False if rate_code == 1 else True,
                      "indexFrom": "420000",
                      "deliveryTypeFrom": pickup_variant,
                      "indicesTo": ["101000"],
                      "deliveryTypeTo": "ANY",
                      "masses": [math.ceil(x * 1000) for x in weights]}  # вес переводится в граммы
        resp = session.post(url=url, data=json.dumps(input_data), headers=headers)
        print(resp.text)
        resp.raise_for_status()
        output_data = resp.json()
        print(output_data)

Стандарт от терминала
[{"cardPayment":true,"cashPayment":true,"completenessChecking":"15,25 ₽","courierRate":"от 300₽ за сбор партии","deliveryRate":60804.0,"deliveryTimeMax":0,"deliveryTimeMin":0,"deliveryType":"Почтовое отделение","easyReturn":"от 180₽ за отправление","indexTo":"101000","insuranceValueRate":" от 0,42% от указанной стоимости","mailType":"POSTAL_PARCEL","mailTypeText":"Посылка","mass":5500.0,"massMax":"20 кг","prePostalPreparationRate":"15,25₽ за отправление","sizeMax":"Одна сторона не более 60 см, сумма всех сторон не более 120 см","smsNotificationRate":"7 ₽"},{"cardPayment":false,"cashPayment":false,"deliveryTimeMax":0,"deliveryTimeMin":0,"error":"Расчет тарифов. [TariffErrorJson(code=2003, msg=Посылка 1 класса (47030) от 09.05.2024. Плата за доставку посылки 1 класса для корпоративных клиентов (2580). Физический вес единичного отправления посылка 1 класса для корпоративных клиентов не может превышать 5 кг (2.194). (1.3).)]","indexTo":"101000","mailType":"PARCEL_CLAS